In [11]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from PIL import Image

In [12]:
class Image_(Dataset):
    def __init__(self, images, labels, transform):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [13]:
train_queen = np.load('D:/1 Ép bê tê/5 Kỳ 5 DE/DPL302m/Code/Data_project/mfcc_queen_train.npy')
train_nonqueen = np.load('D:/1 Ép bê tê/5 Kỳ 5 DE/DPL302m/Code/Data_project/mfcc_NonQueen_train.npy')

val_queen = np.load('D:/1 Ép bê tê/5 Kỳ 5 DE/DPL302m/Code/Data_project/mfcc_queen_val.npy')
val_nonqueen = np.load('D:/1 Ép bê tê/5 Kỳ 5 DE/DPL302m/Code/Data_project/mfcc_NonQueen_val.npy')

test_queen  = np.load('D:/1 Ép bê tê/5 Kỳ 5 DE/DPL302m/Code/Data_project/mfcc_queen_test.npy')
test_nonqueen = np.load('D:/1 Ép bê tê/5 Kỳ 5 DE/DPL302m/Code/Data_project/mfcc_nonqueen_test.npy')

In [14]:
transform = transforms.Compose([
    transforms.ToTensor()
])


In [15]:
X_train = np.vstack((train_queen, train_nonqueen))
labels_train = np.concatenate((np.ones(7000), np.zeros(7000)))

In [16]:
X_val = np.vstack((val_queen, val_nonqueen))
labels_val = np.concatenate((np.ones(2000), np.zeros(2000)))

In [17]:
X_test = np.vstack((test_queen, test_nonqueen))
labels_test = np.vstack((np.ones(1000), np.zeros(1000)))

In [18]:
train_set = Image_(X_train, labels_train, transform=transform)
val_set = Image_(X_val, labels_val, transform=transform)
test_set = Image_(X_test, labels_test,  transform=transform)

train_dataloader = DataLoader(train_set, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_set, batch_size=16, shuffle=False)

In [19]:
import torchvision.models as models

class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ModifiedAlexNet, self).__init__()
        self.alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
        self.alexnet.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        num_ftrs = self.alexnet.classifier[6].in_features
        self.alexnet.classifier[6] = nn.Linear(num_ftrs, num_classes)
        nn.init.xavier_uniform_(self.alexnet.classifier[6].weight)

    def forward(self, x):
        x = self.alexnet(x)
        return nn.functional.softmax(x, dim=1)

model = ModifiedAlexNet(num_classes=2)

# Define the criterion and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
conv_params = [param for name, param in model.named_parameters() if 'classifier.6' not in name]
fc_params = [param for name, param in model.named_parameters() if 'classifier.6' in name]

conv_lr = 0.0001
fc_lr = 0.001

optimizer = optim.Adam([
    {'params': conv_params, 'lr': conv_lr},
    {'params': fc_params, 'lr': fc_lr},
])


In [21]:
import torch
from tqdm import tqdm

num_epochs = 20
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_train_samples = 0
    for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        labels = labels.long()

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        running_loss += loss.item()

        # Accuracy
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
        total_train_samples += labels.size(0)

    train_loss = running_loss / len(train_dataloader)
    train_accuracy = running_corrects.double() / total_train_samples

    # Validation
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    total_val_samples = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation"):
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()

            # Accuracy
            _, val_preds = torch.max(val_outputs, 1)
            val_running_corrects += torch.sum(val_preds == val_labels.data)
            total_val_samples += val_labels.size(0)

    val_loss = val_running_loss / len(val_dataloader)
    val_accuracy = val_running_corrects.double() / total_val_samples

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


Epoch 1:   0%|          | 0/875 [00:00<?, ?it/s]

Epoch 1:  84%|████████▍ | 737/875 [20:23<03:49,  1.66s/it]


KeyboardInterrupt: 